# Universidad del Valle de Guatemala
## Laboratorio 9
* Marco Ramirez 19588
* Jose Javier Hurtarte 19707
* Christian Perez 19710

In [79]:
import pandas as pd
import numpy as np
import whois
import clasificador as cl
import sys
from flare.data_science.features import domain_tld_extract

In [83]:
#read large_eve.json and create a dataframe
df = pd.read_json('large_eve.json', lines=True)
df


,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,proto,...,vars,flow,icmp_type,icmp_code,tcp,smtp,email,app_proto_tc,app_proto_ts,stats
0,2017-07-22T17:33:15.784100-0500,2.835707e+14,18519.0,alert,130.0,192.168.203.200,2328.0,192.181.145.109,445.0,TCP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-07-22T17:33:16.571273-0500,1.519048e+15,21531.0,ssh,140.0,192.168.204.60,62990.0,192.168.26.254,22.0,TCP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017-07-22T17:33:16.661646-0500,1.327836e+15,22269.0,dns,110.0,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,59680.0,2001:0500:0001:0000:0000:0000:803f:0235,53.0,UDP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017-07-22T17:33:17.561063-0500,9.632021e+14,28731.0,http,140.0,192.168.204.59,38011.0,192.168.24.201,80.0,TCP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017-07-22T17:33:17.595416-0500,9.632021e+14,28994.0,fileinfo,140.0,192.168.24.201,80.0,192.168.204.59,38011.0,TCP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
746904,2017-07-22T19:47:49.301548-0500,1.970284e+15,NaN,flow,NaN,192.168.203.200,2007.0,192.183.45.157,445.0,TCP,...,NaN,"{'pkts_toserver': 2, 'pkts_toclient': 0, 'byte...",NaN,NaN,"{'tcp_flags': '02', 'tcp_flags_ts': '02', 'tcp...",NaN,NaN,NaN,NaN,NaN
746905,2017-07-22T19:47:49.301548-0500,2.251792e+15,NaN,flow,NaN,192.168.198.57,52498.0,192.168.21.141,22.0,TCP,...,NaN,"{'pkts_toserver': 2, 'pkts_toclient': 0, 'byte...",NaN,NaN,"{'tcp_flags': '02', 'tcp_flags_ts': '02', 'tcp...",NaN,NaN,NaN,NaN,NaN
746906,2017-07-22T19:47:49.301548-0500,8.444188e+14,NaN,flow,NaN,192.168.201.72,57805.0,192.168.26.225,80.0,TCP,...,NaN,"{'pkts_toserver': 6, 'pkts_toclient': 6, 'byte...",NaN,NaN,"{'tcp_flags': '1b', 'tcp_flags_ts': '1b', 'tcp...",NaN,NaN,NaN,NaN,NaN
746907,2017-07-22T19:47:49.301548-0500,2.814688e+14,NaN,flow,NaN,192.168.201.72,43004.0,192.168.26.225,80.0,TCP,...,NaN,"{'pkts_toserver': 6, 'pkts_toclient': 6, 'byte...",NaN,NaN,"{'tcp_flags': '1b', 'tcp_flags_ts': '1b', 'tcp...",NaN,NaN,NaN,NaN,NaN


In [84]:
#show all columns of df
df.columns

Index(['timestamp', 'flow_id', 'pcap_cnt', 'event_type', 'vlan', 'src_ip',
       'src_port', 'dest_ip', 'dest_port', 'proto', 'alert', 'ssh', 'dns',
       'tx_id', 'http', 'app_proto', 'fileinfo', 'tls', 'vars', 'flow',
       'icmp_type', 'icmp_code', 'tcp', 'smtp', 'email', 'app_proto_tc',
       'app_proto_ts', 'stats'],
      dtype='object')

In [85]:
df['proto'].unique()

array(['TCP', 'UDP', 'IPv6-ICMP', nan], dtype=object)

In [86]:
df_dns=df[df['dns'].notna()]
df_dns

,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,proto,...,vars,flow,icmp_type,icmp_code,tcp,smtp,email,app_proto_tc,app_proto_ts,stats
2,2017-07-22T17:33:16.661646-0500,1.327836e+15,22269.0,dns,110.0,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,59680.0,2001:0500:0001:0000:0000:0000:803f:0235,53.0,UDP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38,2017-07-22T17:33:24.990320-0500,2.022925e+15,54352.0,dns,110.0,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,38051.0,2001:0500:0003:0000:0000:0000:0000:0042,53.0,UDP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39,2017-07-22T17:33:27.379891-0500,5.785448e+14,54519.0,dns,150.0,192.168.205.170,31393.0,192.168.207.4,53.0,UDP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40,2017-07-22T17:33:27.380146-0500,5.785448e+14,54520.0,dns,150.0,192.168.207.4,53.0,192.168.205.170,31393.0,UDP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41,2017-07-22T17:33:27.380146-0500,5.785448e+14,54520.0,dns,150.0,192.168.207.4,53.0,192.168.205.170,31393.0,UDP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
716752,2017-07-22T19:47:03.373435-0500,8.918761e+14,4441474.0,dns,120.0,192.168.207.4,53.0,192.168.202.172,60654.0,UDP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
716753,2017-07-22T19:47:03.373435-0500,8.918761e+14,4441474.0,dns,120.0,192.168.207.4,53.0,192.168.202.172,60654.0,UDP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
716893,2017-07-22T19:47:38.146503-0500,2.199316e+15,4452917.0,dns,130.0,192.168.203.67,50975.0,192.168.207.4,53.0,UDP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
716894,2017-07-22T19:47:38.146953-0500,2.199316e+15,4452918.0,dns,130.0,192.168.207.4,53.0,192.168.203.67,50975.0,UDP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [87]:
df_dns.head(2)

,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,proto,...,vars,flow,icmp_type,icmp_code,tcp,smtp,email,app_proto_tc,app_proto_ts,stats
2,2017-07-22T17:33:16.661646-0500,1.327836e+15,22269.0,dns,110.0,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,59680.0,2001:0500:0001:0000:0000:0000:803f:0235,53.0,UDP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38,2017-07-22T17:33:24.990320-0500,2.022925e+15,54352.0,dns,110.0,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,38051.0,2001:0500:0003:0000:0000:0000:0000:0042,53.0,UDP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [88]:

#filter DGA in DNS 
df_dns_dga= pd.json_normalize(df_dns['dns'])
df_dns_dga

,type,id,rrname,rrtype,tx_id,rcode,ttl,rdata
0,query,15529,api.wunderground.com,A,0.0,NaN,NaN,NaN
1,query,58278,stork79.dropbox.com,A,0.0,NaN,NaN,NaN
2,query,54724,hpca-tier2.office.aol.com.ad.aol.aoltw.net,A,0.0,NaN,NaN,NaN
3,answer,54724,hpca-tier2.office.aol.com.ad.aol.aoltw.net,NaN,NaN,NXDOMAIN,NaN,NaN
4,answer,54724,<root>,SOA,NaN,NXDOMAIN,20864.0,NaN
...,...,...,...,...,...,...,...,...
15744,answer,33107,en-us.start3.mozilla.com,NaN,NaN,NXDOMAIN,NaN,NaN
15745,answer,33107,<root>,SOA,NaN,NXDOMAIN,20864.0,NaN
15746,query,35892,whitecell.localdomain,AAAA,0.0,NaN,NaN,NaN
15747,answer,35892,whitecell.localdomain,NaN,NaN,NXDOMAIN,NaN,NaN


In [89]:
df_dns_a=df_dns_dga[df_dns_dga['rrtype']=='A']
df_dns_a

,type,id,rrname,rrtype,tx_id,rcode,ttl,rdata
0,query,15529,api.wunderground.com,A,0.0,NaN,NaN,NaN
1,query,58278,stork79.dropbox.com,A,0.0,NaN,NaN,NaN
2,query,54724,hpca-tier2.office.aol.com.ad.aol.aoltw.net,A,0.0,NaN,NaN,NaN
5,query,45082,api.wunderground.com,A,0.0,NaN,NaN,NaN
6,query,7425,safebrowsing.clients.google.com.home,A,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
15713,query,15886,secure.informaction.com.hsd1.pa.comcast.net,A,0.0,NaN,NaN,NaN
15716,query,33079,192.168.22.201:.stayonline.net,A,0.0,NaN,NaN,NaN
15725,query,13068,wpad.aol.aoltw.net,A,0.0,NaN,NaN,NaN
15737,query,59436,client-software.real.com,A,0.0,NaN,NaN,NaN


In [90]:
df_unicos=df_dns_a.drop_duplicates(subset=['rrname'])
df_unicos


,type,id,rrname,rrtype,tx_id,rcode,ttl,rdata
0,query,15529,api.wunderground.com,A,0.0,NaN,NaN,NaN
1,query,58278,stork79.dropbox.com,A,0.0,NaN,NaN,NaN
2,query,54724,hpca-tier2.office.aol.com.ad.aol.aoltw.net,A,0.0,NaN,NaN,NaN
6,query,7425,safebrowsing.clients.google.com.home,A,0.0,NaN,NaN,NaN
9,query,12854,fxfeeds.mozilla.com,A,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
15632,query,10109,ntp.ubuntu.com.localdomain,A,0.0,NaN,NaN,NaN
15661,query,46237,redir.metaservices.microsoft.com,A,0.0,NaN,NaN,NaN
15701,query,54310,ocsp.verisign.com,A,0.0,NaN,NaN,NaN
15716,query,33079,192.168.22.201:.stayonline.net,A,0.0,NaN,NaN,NaN


In [91]:
domains = df_unicos['rrname'].tolist()

In [92]:
domains = [domain_tld_extract(domain) for domain in domains]

In [93]:
len(domains)

177

In [94]:
domain_to_delete = [
    '192.168.22.110phpmyadmin',
    '192.168.22.110phpmyadmin.localdomain',
    '"192.168.206.56"',
    '192.168.26-27.0',
    '192.168.21.1201',
    '1922.168.22.254',
    '1922.168.22.254.home',
    '192.168.21-28.0.home',
    '192.168.22.201:',
    '192.168.22.201:.stayonline.net',
    ' ',
    '',
]

new_domains = [domain for domain in domains if domain.strip() not in domain_to_delete]
unique_new_domains = list(set(new_domains))

In [95]:
len(unique_new_domains)

102

In [96]:
domains_df = pd.DataFrame(unique_new_domains, columns=['domain_tld'])
classified = cl.clasificacion(domains_df)

c:\Users\Christian\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.0.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [97]:
sospechosos = classified[classified['isDGA'] == 1]

In [98]:
len(sospechosos)

35

In [99]:
cisco_umbrella = pd.read_csv('top-1m.csv', header=None)

In [100]:
cisco_umbrella = cisco_umbrella[1].to_list()

In [101]:
cisco_umbrella[:10]

['netflix.com',
 'api-global.netflix.com',
 'prod.netflix.com',
 'push.prod.netflix.com',
 'google.com',
 'microsoft.com',
 'www.google.com',
 'safebrowsing.googleapis.com',
 'doubleclick.net',
 'ftl.netflix.com']

In [102]:
non_sus = [domain for domain in sospechosos['domain_tld'].tolist() if domain in cisco_umbrella]
sus = [domain for domain in sospechosos['domain_tld'].tolist() if domain not in cisco_umbrella]

In [103]:
print('No sospechosos: ', len(non_sus))
print('Sospechosos: ', len(sus))

No sospechosos:  25
Sospechosos:  10


In [107]:
sus

['malwarecity.com',
 'ntp.ubuntu.com.localdomain',
 'ecvps.com',
 '192.168.21-28.0',
 'ejfodfmfxlkgifuf.xyz',
 'aoltw.net',
 'bigflickrfeed.com',
 'sql-ledger.org',
 'vtlfccmfxlkgifuf.com',
 'ntkrnlpa.info']

In [108]:
date = []

for n in sus:
    try:
        fecha = whois.whois(n)
        date.append({
            'domain': n,
            'fecha': fecha.creation_date
        })
    except:
        date.append({
            'domain': n,
            'fecha': 'No se pudo obtener'
        })


In [109]:
for n in date:
    print(n)

{'domain': 'malwarecity.com', 'fecha': [datetime.datetime(2008, 2, 6, 15, 42, 35), datetime.datetime(2008, 2, 6, 0, 0)]}
{'domain': 'ntp.ubuntu.com.localdomain', 'fecha': None}
{'domain': 'ecvps.com', 'fecha': datetime.datetime(2009, 5, 21, 14, 24, 2)}
{'domain': '192.168.21-28.0', 'fecha': None}
{'domain': 'ejfodfmfxlkgifuf.xyz', 'fecha': None}
{'domain': 'aoltw.net', 'fecha': datetime.datetime(2000, 1, 10, 4, 9, 19)}
{'domain': 'bigflickrfeed.com', 'fecha': 'No se pudo obtener'}
{'domain': 'sql-ledger.org', 'fecha': datetime.datetime(2000, 9, 8, 16, 25, 9)}
{'domain': 'vtlfccmfxlkgifuf.com', 'fecha': 'No se pudo obtener'}
{'domain': 'ntkrnlpa.info', 'fecha': datetime.datetime(2021, 9, 20, 18, 1, 53)}


Los dominios que cumplen con ser un dominio DGA son: **ejfodfmfxlkgifuf.xyz** y **vtlfccmfxlkgifuf.com**